### QED-C Benchmarks - Qiskit - Hydrogen Lattice - Extras

The notebook contains additional examples specific to this benchmark program.

In [ ]:
min_qubits=2
max_qubits=8
max_circuits=1
num_shots=1000

backend_id="qasm_simulator"
hub="ibm-q"; group="open"; project="main"
provider_backend = None
exec_options = {}

# # *** If using IBMQ hardware, run this once to authenticate
# from qiskit import IBMQ
# IBMQ.save_account('YOUR_API_TOKEN_HERE')

# # *** If you are part of an IBMQ group, set hub, group, and project name here
# hub="YOUR_HUB_NAME"
# group="YOUR_GROUP_NAME"
# project="YOUR_PROJECT_NAME"

# # *** This example shows how to specify the backend using a known "backend_id"
# # Use 'sabre' layout for IBM backends
# exec_options = { "optimization_level":3, "layout_method":'sabre', "routing_method":'sabre',
#                   "use_sessions":True, "resilience_level":1}
# backend_id="ibmq_belem"

# # *** Here's an example of using a typical custom provider backend (e.g. AQT simulator)
# import os
# from qiskit_aqt_provider import AQTProvider
# provider = AQTProvider(os.environ.get('AQT_ACCESS_KEY'))    # get your key from environment
# provider_backend = provider.backends.aqt_qasm_simulator
# backend_id="aqt_qasm_simulator"

# # An example using IonQ provider
# from qiskit_ionq import IonQProvider
# provider = IonQProvider()   # Be sure to set the QISKIT_IONQ_API_TOKEN environment variable
# provider_backend = provider.get_backend("ionq_qpu")
# backend_id="ionq_qpu"

# # An example using BlueQubit provider
# import sys
# sys.path.insert(1, "../..")
# import os, bluequbit, _common.executors.bluequbit_executor as bluequbit_executor
# provider_backend = bluequbit.init()
# backend_id="BlueQubit-CPU"
# exec_options = { "executor": bluequbit_executor.run, "device":'cpu' }


### Load Data From File and Regenerate Plots
This cell can be used to re-load the data stored during benchmark execution and plot the results.

Be sure to first execute the initial cell which sets the backend_id

In [ ]:
import os, hydrogen_lattice_benchmark

#backend_id="qasm_simulator"

# Additional arguments specific to Hydrogen Lattice benchmark method 2 plotting
hl_app_args = dict(
    
    # display options for line plots (pairwise)
    line_y_metrics=['energy', 'accuracy_ratio_error'],             # + 'solution_quality', 'accuracy_ratio', 'solution_quality_error'
    line_x_metrics=['iteration_count', 'cumulative_exec_time'],    # + 'cumulative_elapsed_time'
    plot_layout_style='grid',                                      # plot layout, can be 'grid', 'stacked', or 'individual'
    
    # display options for bar plots (exec time, accuracy ratio)
    use_logscale_for_times=False,                                  # use log scale for cumulative exec time bar chart
    
    # display options for area plots (multiplicative)
    score_metric=['accuracy_ratio'],                               # + 'solution_quality'
    x_metric=['cumulative_exec_time', 'cumulative_elapsed_time'],  # + 'cumulative_opt_exec_time', 
)

hydrogen_lattice_benchmark.load_data_and_plot(os.path.join('__data', backend_id, ''), backend_id=backend_id,                                             
    **hl_app_args)


### Hydrogen Lattice - Method 2 - Use Custom Minimizer Function

This example shows how to configure a custom minimizer function for use in this benchmark.


In [ ]:
import sys
sys.path.insert(1, "hydrogen-lattice/qiskit")
import hydrogen_lattice_benchmark

# define a custom Nelder-Mead minimizer function
from scipy.optimize import minimize

tol=0.01
max_iter=30

def my_minimizer(objective_function, initial_parameters, callback):

    ret = minimize(objective_function,
        x0=initial_parameters, 

        
        # a custom minimizer 
        method='nelder-mead',
        options={'xatol':tol, 'fatol':tol, 'maxiter': max_iter, 'maxfev': max_iter, 'disp': False},
                   
        callback=callback) 
    
    print(f"\n... my_minimizer completed, return = \n{ret}")
    return ret

# Additional arguments specific to Hydrogen Lattice benchmark method 2
hl_app_args = dict(   

    max_iter=30,                # maximum minimizer iterations to perform
    comfort=True,               # show 'comfort dots' during execution
   
    minimizer_function=my_minimizer,   # use custom minimizer function
)

# Run the benchmark in method 2
hydrogen_lattice_benchmark.run(
    min_qubits=min_qubits, max_qubits=max_qubits, max_circuits=max_circuits, num_shots=num_shots,
    method=2, 
    backend_id=backend_id, provider_backend=provider_backend,
    hub=hub, group=group, project=project, exec_options=exec_options,
    **hl_app_args
)